In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.0'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-18 18:20:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-08-18 18:20:26 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

from pyspark import SparkFiles
# Load file
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz"

In [4]:
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_multilingual_JP_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------+-------------------------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|                   product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|                review_headline|                          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------+-------------------------------------+-----------+
|         JP|      65317|R33RSUD4ZTRKT7|B000001GBJ|     957145596|            SONGS FROM A SECR...|           Music|          1|            1|         15|   N|                Y|                    残念ながら…| 残念ながら…趣味ではありません

In [5]:
# Count the number of records (rows) in the dataset
rows = df.count()
print(f"dataframe has {rows} rows")

dataframe has 262431 rows


In [6]:
# Transform the dataset to fit the tables in the schema file. Be sure that the DataFrames match in data type and in column name.

#check for null values - we don't want any in review_id, customer_id, or product_id.
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               0|          0|            0|          0|   0|                0|              1|          3|          8|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [7]:
# Get column names and types
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:
# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );

from pyspark.sql.functions import col
from pyspark.sql.functions import to_date

review_id_table = df[['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']]
review_id_table = review_id_table.withColumn("review_date",to_date(col("review_date"),"yyyy-mm-dd"))
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R33RSUD4ZTRKT7|      65317|B000001GBJ|     957145596| 2012-01-05|
|R2U1VB8GPZBBEH|      65317|B000YPWBQ2|     904244932| 2012-01-05|
|R1IBRCJPPGWVJW|      65696|B0002E5O9G|     108978277| 2013-01-02|
| RL02CW5XLYONU|      67162|B00004SRJ5|     606528497| 2013-01-11|
|R2LA2SS3HU3A3L|      67701|B0093H8H8I|     509738390| 2013-01-10|
|R32DR19C0UCHYN|      68380|B00712Y28Q|      37188049| 2014-01-09|
|R3O2A01NE9JI2O|      68655|B00N0YIQFE|     916594821| 2015-01-08|
| RH1BDBT513AMD|      68973|B0015HOFZI|      29559888| 2012-01-30|
|R1J820Q5MFGUVX|      69080|B001R0KH3M|     369834710| 2014-01-19|
|R1A63XC56E0M30|      69552|B007ZB06OW|     380169630| 2013-01-06|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [9]:
# -- This table will contain only unique values
# CREATE TABLE products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );

products = df[['product_id', 'product_title']]
products = products.dropDuplicates()
products.count()

38447

In [10]:
products.show(10)

+----------+------------------------------------+
|product_id|                       product_title|
+----------+------------------------------------+
|B0002E5O9G|                Les Miserables 10...|
|B00NH13W2W|           Amazonベーシック Apple...|
|B00REIAUD8|                 Love Me Like You Do|
|B00JGI5NEK|                Joint Tenants (En...|
|B00NT5NC5I| ホエア・ウィー・アー' ライブ・フ...|
|B00MOBNRY4|       レゴ (LEGO) アーキテクチャ...|
|B003CN5W1E|  バイオハザードIV アフターライフ...|
|B000816TCY|           Nikon Li-ionリチャージ...|
|1465418970|                LEGO Star Wars: T...|
|B00LISORJI|ゲイリー・ムーア~ライヴ・アット・...|
+----------+------------------------------------+
only showing top 10 rows



In [11]:
# -- Customer table for first data set
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );

customers = df[['customer_id']]
customers = customers.groupBy("customer_id").count()
customers = customers.withColumnRenamed("count","customer_count")

customers.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    6101092|             1|
|    6284204|             1|
|    6463589|             1|
|    6483599|             1|
|    7135090|             1|
|    7144452|             1|
|    7156796|             1|
|    7249117|             1|
|    7336981|             1|
|    7469928|             1|
+-----------+--------------+
only showing top 10 rows



In [12]:
# -- vine table
# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );



vine_table = df[['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']]
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R33RSUD4ZTRKT7|          1|            1|         15|   N|
|R2U1VB8GPZBBEH|          1|            4|         20|   N|
|R1IBRCJPPGWVJW|          5|            2|          3|   N|
| RL02CW5XLYONU|          5|            6|          9|   N|
|R2LA2SS3HU3A3L|          4|            2|          4|   N|
|R32DR19C0UCHYN|          4|            2|          3|   N|
|R3O2A01NE9JI2O|          5|            8|         13|   N|
| RH1BDBT513AMD|          1|            3|         15|   N|
|R1J820Q5MFGUVX|          5|            1|          2|   N|
|R1A63XC56E0M30|          4|            1|          4|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [13]:
#Load the DataFrames that correspond to tables into an RDS instance. 
# Note: This process can take up to 10 minutes for each. Ensure that everything is correct before uploading.

 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://aws-reviews.cxtrj4clfzmw.us-east-1.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"root", 
          "password": "BeergutBobby", 
          "driver":"org.postgresql.Driver"}

          ## REMEMBER TO DELETE PASSWORD and ENDPOINT AFORE UPLOADING
          ## AND THEN SHUT DOWN ALL AWS BUCKETS AND RDS

In [ ]:
# Write DataFrames to tables

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)